In [ ]:
!pip install keras-metrics

In [ ]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import keras_metrics as km
 
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
data_dir = "/content/drive/MyDrive/Classification/data_another/"
img_height , img_width = 200, 200
seq_len = 100
 
classes = ["nw","ww"]

In [ ]:
def frames_extraction(video_path):
    frames_list = []
     
    vidObj = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
 
    while count <= seq_len: 
         
        success, image = vidObj.read() 
        if success:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            count += 1
        else:
            print("Defected frame")
            break
 
            
    return frames_list

In [ ]:
def create_data(input_dir):
    X = []
    Y = []
    ff=0
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
          ff=ff+1
          frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
          if len(frames) == seq_len:
                X.append(frames)
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    print(ff)
    return X, Y

In [ ]:
X, Y = create_data(data_dir)
 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=0)
 
model = Sequential()
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(2, activation = "softmax"))
 
model.summary()

nw
Defected frame
ww
12
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, 62, 62, 64)        154624    
_________________________________________________________________
dropout_2 (Dropout)          (None, 62, 62, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 246016)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               62980352  
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 514       
Total params: 63,135,490
Trainable params: 63,135,490
Non-trainable params: 0
__________________

In [ ]:
print(X_train.shape)

(8, 100, 64, 64, 3)


In [ ]:
print(y_train.shape)

(8, 2)


In [ ]:
opt = keras.optimizers.SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
 
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]
 
history = model.fit(x = X_train, y = y_train, epochs=40, batch_size = 1 , shuffle=True, validation_split=0.2, callbacks=callbacks)
 


Epoch 1/40
6/6 [==============================] - 65s 11s/step - loss: 2.4409 - accuracy: 0.8405 - val_loss: 8.4826 - val_accuracy: 0.5000
Epoch 2/40
6/6 [==============================] - 62s 10s/step - loss: 6.7040 - accuracy: 0.4667 - val_loss: 5.7258 - val_accuracy: 0.5000
Epoch 3/40
6/6 [==============================] - 61s 10s/step - loss: 5.9790 - accuracy: 0.4214 - val_loss: 6.7745 - val_accuracy: 0.5000
Epoch 4/40
6/6 [==============================] - 62s 10s/step - loss: 0.8383 - accuracy: 0.9238 - val_loss: 1.9412 - val_accuracy: 0.5000
Epoch 5/40
6/6 [==============================] - 62s 10s/step - loss: 5.2349 - accuracy: 0.5810 - val_loss: 2.5584 - val_accuracy: 0.5000
Epoch 6/40
6/6 [==============================] - 64s 10s/step - loss: 2.0571 - accuracy: 0.4667 - val_loss: 2.6280 - val_accuracy: 0.5000
Epoch 7/40
6/6 [==============================] - 62s 10s/step - loss: 2.7991 - accuracy: 0.4667 - val_loss: 4.1138 - val_accuracy: 0.5000
Epoch 8/40
6/6 [===========

NameError: ignored

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import keras
from keras.layers import Dense, LSTM, \
    Flatten, TimeDistributed, Conv2D, Dropout
from keras import Sequential
from keras.applications.vgg16 import VGG16
# create a VGG16 "model", we will use
# image with shape (224, 224, 3)
vgg = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(200, 200, 3)
)
for layer in vgg.layers[:-4]:
    layer.trainable = False
# create a Sequential model
model_lo = Sequential()
model_lo.add(TimeDistributed(vgg, input_shape=(100, 200, 200, 3)))
model_lo.add(TimeDistributed(Flatten()))
model_lo.add(LSTM(256, activation='relu', return_sequences=False))
model_lo.add(Dense(64, activation='relu'))
model_lo.add(Dropout(.5))
model_lo.add(Dense(2, activation='softmax'))

In [ ]:
model_lo.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 100, 6, 6, 512)    14714688  
_________________________________________________________________
time_distributed_2 (TimeDist (None, 100, 18432)        0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               19137536  
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 130       
Total params: 33,868,802
Trainable params: 26,233,538
Non-trainable params: 7,635,264
__________________________________

In [ ]:
model_lo.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
 
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]

In [ ]:
img_height , img_width = 200, 200
seq_len = 100
X, Y = create_data(data_dir)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=0)

nw
Defected frame
ww
12


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8, 100, 200, 200, 3)
(3, 100, 200, 200, 3)
(8, 2)
(3, 2)


In [ ]:
history = model_lo.fit(x = X_train, y = y_train, epochs=40, batch_size = 1 , shuffle=True, validation_split=0.2, callbacks=callbacks)

Epoch 1/40
6/6 [==============================] - 362s 63s/step - loss: nan - accuracy: 0.5810 - val_loss: nan - val_accuracy: 0.5000
Epoch 2/40
6/6 [==============================] - 357s 62s/step - loss: nan - accuracy: 0.2786 - val_loss: nan - val_accuracy: 0.5000
Epoch 3/40
6/6 [==============================] - 358s 62s/step - loss: nan - accuracy: 0.2786 - val_loss: nan - val_accuracy: 0.5000
Epoch 4/40
6/6 [==============================] - 357s 62s/step - loss: nan - accuracy: 0.6048 - val_loss: nan - val_accuracy: 0.5000
Epoch 5/40
6/6 [==============================] - 358s 62s/step - loss: nan - accuracy: 0.2714 - val_loss: nan - val_accuracy: 0.5000
Epoch 6/40
6/6 [==============================] - 359s 62s/step - loss: nan - accuracy: 0.3905 - val_loss: nan - val_accuracy: 0.5000
Epoch 7/40
3/6 [==============>...............] - ETA: 2:21 - loss: nan - accuracy: 0.2778

In [ ]:
import keras
from keras.layers import Dense, LSTM, \
    Flatten, TimeDistributed, Conv2D, Dropout
from keras import Sequential
from keras.applications.vgg16 import VGG16
from keras.applications import InceptionV3
incep = InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(200, 200, 3)
)
model_lo = Sequential()
model_lo.add(TimeDistributed(incep, input_shape=(100, 200, 200, 3)))
model_lo.add(TimeDistributed(Flatten()))
model_lo.add(LSTM(256, activation='relu', return_sequences=False))
model_lo.add(Dense(64, activation='relu'))
model_lo.add(Dropout(.5))
model_lo.add(Dense(2, activation='softmax'))

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
model_lo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 100, 4, 4, 2048)   21802784  
_________________________________________________________________
time_distributed_1 (TimeDist (None, 100, 32768)        0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               33817600  
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 55,636,962
Trainable params: 55,602,530
Non-trainable params: 34,432
_______________________________________